In [2]:
#Import packages
#---------------------------------------
import sys
import os
import pandas as pd
from matplotlib import pyplot as plt
import numpy as np
import matplotlib
import warnings
import random
import arviz as az
import pymc as pm
warnings.filterwarnings("ignore", category=RuntimeWarning) 

#Import your modules
#---------------------------------------
import admin_functions as adfn
import cell_decomp_func as cdfn


# Define paths
#----------------------------------------------------------------------
l_code = '/Users/dominicburrows/Dropbox/PhD/Analysis/my_scripts/GitHub/'
l_data = '/Users/dominicburrows/Dropbox/PhD/analysis/Project/'
l_fig = '/Users/dominicburrows/Dropbox/PhD/figures/'

s_code = '/cndd3/dburrows/CODE/'
s_data = '/cndd3/dburrows/DATA/'
%load_ext autoreload
sys.version

'3.11.0 | packaged by conda-forge | (main, Jan 14 2023, 12:27:40) [GCC 11.3.0]'

In [4]:
rho_list = np.linspace(0,0.9,10)

array([0. , 0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9])

In [12]:
# Run epsilon noise

#Define parameters of simulated data
n_clusts = 5
n_genes = 800
n_cells = 100
rate_range = 0,40 #max and min of uniform distribution for generating rates
model_type = 'epsilon' #CHANGE!
noise_type = 'exponential' #CHANGE!
mode = 'rho'

for x in rho_list: #CHANGE!
    #Simulate spot data from simulated gene expression
    spot_sim = cdfn.simulate_cell_mix(n_clusts, n_cells, n_genes).simulate_gene_exp(rate_range, x)
    n_spots = spot_sim.__dict__['n_spots']
    spots = spot_sim.__dict__['spots']
    ref_exp = spot_sim.__dict__['mean_exps']
    prop_vec = spot_sim.__dict__['prop_vec']
    spots = cdfn.add_noise(spots, per=None, a_std=5, g_std=5, e_std=5, noise_type=noise_type) #CHANGE!

    model = cdfn.epsilon_pymc(n_clusts, n_spots, n_genes, ref_exp, spots, noise_type=noise_type) #CHANGE!
    with model: data=pm.sample(random_seed=1,draws=200,chains=1, discard_tuned_samples=False)

    post, r2 = cdfn.model_stats(data, prop_vec, n_clusts)
    if len(str(x))<2:pref = '0' + str(x)
    else:pref= str(x)
    np.save(s_data + 'spatial_transcriptomics/RCTD-test_noise-' + noise_type + '_model-' + model_type + '_' + mode + '-' + pref + '.npy', np.array([prop_vec, post, r2], dtype=object))
    print(x)

